In [3]:
import nltk
nltk.download("punkt")
from gensim.models import Word2Vec, FastText

[nltk_data] Downloading package punkt to /Users/jnooree/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
corpus_file = "news.txt"
with open(corpus_file) as f:
    corpus = [nltk.word_tokenize(line) for line in f]

In [5]:
corpus[:3]

[["'탈모", "'", '치료하고', "'여심", "'", '잡는', '방법', '?'],
 [],
 ['시끌벅적한', '연말', '분위기가', '수그러들자', '신묘년', '새해가', '밝았다', '.']]

In [6]:
wv_win2 = Word2Vec(sg=1, window=2)

In [7]:
wv_win2.build_vocab(corpus)
wv_win2.train(corpus, total_examples=wv_win2.corpus_count, epochs=wv_win2.epochs)

(41506820, 52590300)

In [8]:
wv_win2.wv.most_similar('탈모')

[('여성탈모', 0.8192265033721924),
 ('탈모의', 0.8150723576545715),
 ('비만', 0.7978000640869141),
 ('질염', 0.7891401052474976),
 ('탈모는', 0.7875183820724487),
 ('남성형', 0.7760628461837769),
 ('갱년기', 0.7690579891204834),
 ('변비', 0.7690173983573914),
 ('전립선', 0.7671183347702026),
 ('치매', 0.7668051719665527)]

In [9]:
wv_win5 = Word2Vec(sentences=corpus, sg=1, window=5)

In [10]:
wv_win2.wv.most_similar(positive=['한국', '도쿄'], negative=['일본'])

[('도시인', 0.7480379939079285),
 ('베이징', 0.7399730086326599),
 ('메카인', 0.7339820861816406),
 ('상하이', 0.7291325926780701),
 ('동대문', 0.7275378704071045),
 ('피겨스케이팅', 0.7263103127479553),
 ('스케이팅', 0.722897469997406),
 ('명문', 0.7212512493133545),
 ('칸', 0.7182360291481018),
 ('베를린', 0.716895580291748)]

In [11]:
ft = FastText(sentences=corpus, sg=1)

In [13]:
ft.wv.most_similar('탈모')

[('탈모란', 0.9131588935852051),
 ('탈모나', 0.9024038314819336),
 ('탈모도', 0.8924594521522522),
 ('탈모의', 0.8808624148368835),
 ('탈모와', 0.8746344447135925),
 ('탈모예방', 0.860072135925293),
 ('탈모로', 0.8532431125640869),
 ('탈모가', 0.8512105941772461),
 ('탈모를', 0.8420150279998779),
 ('탈모증의', 0.8406862020492554)]

In [12]:
ft.wv.most_similar('한국')

[('한국의', 0.8112791776657104),
 ('한국뿐', 0.7942297458648682),
 ('한국팀', 0.7841842770576477),
 ('한국영', 0.7799909114837646),
 ('한국화', 0.7776421308517456),
 ('한국식', 0.7765466570854187),
 ('한국선급', 0.7739657163619995),
 ('아시아', 0.7720630168914795),
 ('중국', 0.7720608711242676),
 ('한국서', 0.7699242830276489)]

In [14]:
wv_win2.wv.similarity('호랑이', '고양이')

0.7787038

In [15]:
wv_win2.wv.similarity('왕', '배추')

0.3970073

In [16]:
import pandas as pd

In [17]:
data = pd.read_csv("WS353_korean.csv")
data

,word 1,word 2,kor_score
0,사랑,섹스,6.42
1,호랑이,고양이,7.17
2,호랑이,호랑이,10.00
3,책,종이,6.17
4,컴퓨터,키보드,6.67
...,...,...,...
348,소나기,홍수,6.25
349,날씨,예보,6.42
350,재난,지역,5.25
351,도지사,사무실,2.92


In [18]:
def get_sim(model, words):
    try:
        return model.wv.similarity(words[0], words[1])
    except KeyError:
        return

In [19]:
get_sim(ft, ['호랑이', '고양이'])

0.7627694

In [23]:
data['wv_win2'] = data.loc[:, 'word 1':'word 2'].apply(lambda words: get_sim(wv_win2, words), axis=1)

In [24]:
data

,word 1,word 2,kor_score,wv_win2
0,사랑,섹스,6.42,0.434523
1,호랑이,고양이,7.17,0.778704
2,호랑이,호랑이,10.00,1.000000
3,책,종이,6.17,0.556271
4,컴퓨터,키보드,6.67,0.698369
...,...,...,...,...
348,소나기,홍수,6.25,0.471163
349,날씨,예보,6.42,0.512973
350,재난,지역,5.25,0.454408
351,도지사,사무실,2.92,0.401895


In [25]:
data.loc[:, 'kor_score':'wv_win2'].corr()

,kor_score,wv_win2
kor_score,1.000000,0.315857
wv_win2,0.315857,1.000000


In [26]:
data['wv_win5'] = data.loc[:, 'word 1':'word 2'].apply(lambda words: get_sim(wv_win5, words), axis=1)

In [27]:
data.loc[:, 'kor_score':'wv_win5'].corr()

,kor_score,wv_win2,wv_win5
kor_score,1.000000,0.315857,0.387332
wv_win2,0.315857,1.000000,0.931126
wv_win5,0.387332,0.931126,1.000000
